In [64]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


pd.set_option('display.max_colwidth', None)

In [65]:
similarity_path = "../data/processed/vis_dataset/similarity_combined.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "../reports/figures/ref_similarity/similarity_using_citeTo_and_citedBy.html"

In [66]:
dataset = pd.read_csv(dataset_path)

dataset["Title"] = dataset["Title"].astype(str)
dataset["AuthorNames"] = dataset["AuthorNames"].astype(str)


dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
1537,SciVis,2018,Visualization of Neuronal Structures in Wide-Field Microscopy Brain Images,10.1109/TVCG.2018.2864852,1537,"[745, 1397, 985, 889, 1279]",[],http://dx.doi.org/10.1109/TVCG.2018.2864852,1018,1028,...,"Wide-field microscopes are commonly used in neurobiology for experimental studies of brain samples. Available visualization tools are limited to electron, two-photon, and confocal microscopy datasets, and current volume rendering techniques do not yield effective results when used with wide-field data. We present a workflow for the visualization of neuronal structures in wide-field microscopy images of brain samples. We introduce a novel gradient-based distance transform that overcomes the out-of-focus blur caused by the inherent design of wide-field microscopes. This is followed by the extraction of the 3D structure of neurites using a multi-scale curvilinear filter and cell-bodies using a Hessian-based enhancement filter. The response from these filters is then applied as an opacity map to the raw data. Based on the visualization challenges faced by domain experts, our workflow provides multiple rendering modes to enable qualitative analysis of neuronal structures, which includes separation of cell-bodies from neurites and an intensity-based classification of the structures. Additionally, we evaluate our visualization results against both a standard image processing deconvolution technique and a confocal microscopy image of the same specimen. We show that our method is significantly faster and requires less computational resources, while producing high quality visualizations. We deploy our workflow in an immersive gigapixel facility as a paradigm for the processing and visualization of large, high-resolution, wide-field microscopy brain datasets.",Saeed Boorboor;Shreeraj Jadhav;Mala Ananth;David Talmage;Lorna Role;Arie E. Kaufman,Saeed Boorboor;Shreeraj Jadhav;Mala Ananth;David Talmage;Lorna Role;Arie Kaufman,Department of Computer ScienceStony Brook University;Department of Computer ScienceStony Brook University;Department of Neurobiology & behaviorStony Brook University;Department of Neurobiology & behaviorStony Brook University;Department of Neurobiology & behaviorStony Brook University;Department of Computer ScienceStony Brook University,10.1109/TVCG.2014.2346312;10.1109/TVCG.2013.142;10.1109/TVCG.2012.203;10.1109/TVCG.2017.2744079;10.1109/TVCG.2009.118;10.1109/TVCG.2016.2598472,"Wide-field microscopy,volume visualization,neuron visualization,neuroscience",NaN,0.0,58.0,NaN
411,VAST,2008,Using visual analytics to maintain situation awareness in astrophysics,10.1109/VAST.2008.4677353,411,"[298, 336, 338, 339, 341, 342, 343, 182, 281]",[862],http://dx.doi.org/10.1109/VAST.2008.4677353,27,34,...,"We present a novel collaborative visual analytics application for cognitively overloaded users in the astrophysics domain. The system was developed for scientists needing to analyze heterogeneous, complex data under time pressure, and then make predictions and time-critical decisions rapidly and correctly under a constant influx of changing data. The Sunfall Data Taking system utilizes several novel visualization and analysis techniques to enable a team of geographically distributed domain specialists to effectively and remotely maneuver a custom-built instrument under challenging operational conditions. Sunfall Data Taking has been in use for over eighteen months by a major international astrophysics collaboration (the largest data volume supernova search currently in operation), and has substantially improved the operational efficiency of its users. We describe the system design process by an interdisciplinary team, the system architecture, and the results of an informal u

In [67]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Authors: </b>: " + dataset["AuthorNames"] + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Authors: </b>: B. Spence;L. Tweedie;H. Dawkes;Hua Su; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken o

In [68]:
df = pd.read_csv(similarity_path)
df

,id,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1742,1742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1743,1743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1744,1744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.0)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','value'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,value
0,0,238,1.0
1,0,244,1.0
2,0,456,1.0
3,0,512,1.0
4,0,550,1.0
...,...,...,...
60213,1611,1612,2.0
60214,1611,1614,1.0
60215,1611,1615,2.0
60216,1614,1615,1.0


## Plot using Pyvis

In [70]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [73]:
thres = 5
df_out_thres = df_out[df_out["value"] >= thres]
plotting_all_nodes = False

file_name= output_path
# g = net.Network(height='95%', width='99%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')

g = net.Network(height="700px", width="100%",heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')
g.set_edge_smooth('discrete')
# g.set_options('{"nodes": {"borderWidthSelected": "10" } }')

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
# g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

g.add_nodes(node_list, title=node_labels, color=node_colors)


edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['value'])
g.add_edges(edge_list)    
# g.show_buttons(filter_=["physics", "edges"])
# g.show_buttons()


g.show(file_name)

#  source code: https://github.com/WestHealth/pyvis/blob/523d541d04df6779dd21e5942af60b79369e8f13/pyvis/network.py#L218

In [74]:
g.num_nodes()

431

In [79]:
import dill

#### Write
with open('/Users/chaupham/minhchau/MY_WORK/C/citation_networks/models/network_plot/citation_similarity.pkl', 'wb') as f:     
    dill.dump(g, f)

In [54]:
file_name

'../reports/figures/ref_similarity/similarity_using_citeTo_and_citedBy.html'

In [50]:
!open $file_name

In [52]:
!open .

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [12]:
dataset

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
0,InfoVis,1995,Visualisation for functional design,10.1109/INFVIS.1995.528680,0,[],[990],http://dx.doi.org/10.1109/INFVIS.1995.528680,4,10,...,"We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken of the uncertainties in parameter values arising from inevitable variations in the manufacturing process.",Robert Spence;Lisa Tweedie;Huw Dawkes;Hua Su,B. Spence;L. Tweedie;H. Dawkes;Hua Su,"Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK;Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK;Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK;Dept. of Electr. Eng., Imperial Coll. of Sci., Technol. & Med., London, UK",NaN,NaN,NaN,12.0,10.0,NaN
1,InfoVis,1995,Towards a generative theory of diagram design,10.1109/INFVIS.1995.528681,1,[],[],http://dx.doi.org/10.1109/INFVIS.1995.528681,11,18,...,"We describe the theoretical background for AVE, an automatic visualization engine for semantic networks. We have a functional notion of aesthetics and therefore understand meaningfulness as a central issue for information visualization. This implies that the diagrams should communicate the characteristics of the data as effectively as possible. In this generative theory of diagram design, we include data characterization, systematic use of graphical means of expression and the combination of graphical means of expression. After giving a brief introduction and an application scenario we discuss these aspects in detail. Finally, a process model of an automatic visualization process is sketched and directions for further research are outlined.",Klaus Reichenberger;Thomas Kamps;Gene Golovchinsky,K. Reichenberger;T. Kamps;G. Golovchinsky,"GMD-Inst. for Integrated Publication & Inf. Sci., Darmstadt, Germany;GMD-Inst. for Integrated Publication & Inf. Sci., Darmstadt, Germany;GMD-Inst. for Integrated Publication & Inf. Sci., Darmstadt, Germany",10.1109/VISUAL.1995.480815,NaN,12.0,4.0,18.0,NaN
2,InfoVis,1995,Research report: information animation applications in the capital markets,10.1109/INFVIS.1995.528682,2,[],"[32, 52]",http://dx.doi.org/10.1109/INFVIS.1995.528682,19,25,...,"3D computer graphics can be extremely expressive. It is possible to display an entire securities market, like the S&amp;P 500, on a single screen. With the correct approach to the visual design of the layout, these massive amounts of information can be quickly and easily comprehended by a human observer. By using motion and animated interaction, it is possible to use 3D as a reliable, accurate and precise decision-support tool. Information animation applications are particularly suited to the securities industry because that is where we find huge amounts of data, the value of which declines rapidly with time, and where critical decisions are being made on this data in very short periods of time. Information animation technology is an important new tool for the securities industry, where people need to be in the decision-making loop without suffering from information overload. Sever

In [ ]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 6
plotting_all_nodes = False


    
    

df_out_thres = df_out[df_out["value"] >= thres]
edges = df_out_thres

(dataset)
# edges.loc[:, 'label'] = edges.loc[:, 'value'].astype(int).astype(str)

edges["from"] = edges["from"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0]))
edges["to"] = edges["to"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 


set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
nodes["Title"] = nodes["id"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 
nodes["id"] = nodes["Title"]
if plotting_all_nodes:
    pass
else:
    print(nodes.shape)
    node_list = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) 
    nodes = nodes[nodes["id"].isin(node_list)]
    print(nodes.shape)

nodes.to_csv("../data/interim/jaal_data/nodes_2.csv", index=False)
edges.to_csv("../data/interim/jaal_data/edges_2.csv", index=False)

port=8050

while True:
    try:
        Jaal(edges, nodes).plot(port=port)
    except:
        port+=1


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(1746, 4)
(294, 4)
Parsing the data...Done
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/chaupham/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2021 00:26:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:26:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:26:53] "GET /_dash-layout HT

No trigger


127.0.0.1 - - [13/Apr/2021 00:26:54] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:26:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 00:27:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.

In [ ]:
edges["value"] = edges["value"].astype(int)

In [ ]:
edges.nlargest(10,"value")

In [ ]:
node_list

In [ ]:
nodes[nodes.id==1606].Title.values[0]

In [ ]:
edges[(edges["to"] == 755) & (edges["from"]==456)]